In [1]:
from __future__ import division
from scipy.stats import gamma
import dowhy
import dowhy.datasets, dowhy.plotter
from dowhy import CausalModel
from dowhy import causal_estimators
import seaborn as sns
import graphviz
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from IPython.display import Image, display
from utility_functions import *

import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)

from numpy import mean
from numpy import std
from numpy.random import randn
from numpy.random import seed
from matplotlib import pyplot

In [2]:
df = pd.read_csv('lucas0_train.csv')
df.head()

,Smoking,Yellow_Fingers,Anxiety,Peer_Pressure,Genetics,Attention_Disorder,Born_an_Even_Day,Car_Accident,Fatigue,Allergy,Coughing,Lung_cancer
0,0,0,1,0,0,1,0,1,0,1,0,0
1,0,1,0,0,0,0,1,0,1,0,1,1
2,1,1,1,0,1,1,1,1,1,1,1,1
3,0,0,0,1,0,0,1,0,0,0,0,0
4,1,1,1,0,0,1,1,1,1,0,0,1


In [3]:
df.describe()

,Smoking,Yellow_Fingers,Anxiety,Peer_Pressure,Genetics,Attention_Disorder,Born_an_Even_Day,Car_Accident,Fatigue,Allergy,Coughing,Lung_cancer
count,2000.000000,2000.000000,2000.00000,2000.000000,2000.000000,2000.00000,2000.000000,2000.000000,2000.000000,2000.00000,2000.000000,2000.000000
mean,0.752500,0.782000,0.63050,0.341500,0.139500,0.32250,0.489500,0.723000,0.737000,0.34300,0.700500,0.721500
std,0.431668,0.412991,0.48279,0.474331,0.346554,0.46755,0.500015,0.447628,0.440373,0.47483,0.458154,0.448373
min,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
25%,1.000000,1.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
50%,1.000000,1.000000,1.00000,0.000000,0.000000,0.00000,0.000000,1.000000,1.000000,0.00000,1.000000,1.000000
75%,1.000000,1.000000,1.00000,1.000000,0.000000,1.00000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000
max,1.000000,1.000000,1.00000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000


### Load Graph

In [4]:
feature_names = list(df.columns)
target_node = 'Lung_cancer'
edges_list = [('Anxiety','Smoking'), ('Peer_Pressure','Smoking'), ('Smoking','Yellow_Fingers'), ('Smoking','Lung_cancer'), 
              ('Genetics','Lung_cancer'), ('Genetics','Attention_Disorder'), ('Allergy','Coughing'), ('Lung_cancer','Coughing'),
              ('Lung_cancer','Fatigue'), ('Coughing','Fatigue'),('Fatigue','Car_Accident'), ('Attention_Disorder','Car_Accident')]

In [5]:
G = nx.DiGraph(directed=True)
G.add_nodes_from(feature_names)
G.add_edges_from(edges_list)

position = {'Anxiety': (-1, 1), 'Peer_Pressure': (0, 1), 'Born_an_Even_Day': (1, 1), 
       'Yellow_Fingers': (-1, 0), 'Smoking': (0, 0),'Genetics': (1, 0), 
       'Allergy': (-1, -1), 'Lung_cancer': (0, -1),'Attention_Disorder': (1, -1),
      'Coughing': (-1, -2), 'Fatigue': (0, -2), 'Car_Accident': (1, -2)}

color_map = ['yellow' if node == target_node else 'w' for node in G]

fig = plt.figure(1, figsize=(10, 8), dpi=60)
nx.draw(G, pos = position, with_labels=True, node_size=6000, node_color=color_map, edgecolors ='black')
nx.write_gml(G, "causal_graph.gml")

In [6]:
def gml_to_string(file):
    gml_str = ''
    with open(file, 'r') as file:
        for line in file:
            gml_str += line.rstrip()
    return gml_str

In [7]:
gml_graph = gml_to_string('causal_graph.gml')

In [8]:
TREATMENT = 'Lung_cancer'
OUTCOME = 'Car_Accident'

In [11]:
model = CausalModel(
                data = df_t,
                treatment = TREATMENT,
                outcome = OUTCOME,
                graph = gml_graph
        )

In [12]:
identified_estimand = model.identify_effect()
print(identified_estimand)

Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
      d                                           
──────────────(Expectation(Car_Accident|Genetics))
d[Lung_cancer]                                    
Estimand assumption 1, Unconfoundedness: If U→{Lung_cancer} and U→Car_Accident then P(Car_Accident|Lung_cancer,Genetics,U) = P(Car_Accident|Lung_cancer,Genetics)

### Estimand : 2
Estimand name: iv
Estimand expression:
Expectation(Derivative(Car_Accident, [Smoking])*Derivative([Lung_cancer], [Smo
king])**(-1))
Estimand assumption 1, As-if-random: If U→→Car_Accident then ¬(U →→{Smoking})
Estimand assumption 2, Exclusion: If we remove {Smoking}→{Lung_cancer}, then ¬({Smoking}→Car_Accident)

### Estimand : 3
Estimand name: frontdoor
Estimand expression:
Expectation(Derivative(Car_Accident, [Fatigue])*Derivative([Fatigue], [Lung_ca
ncer]))
Estimand assumption 1, Full-mediation: Fatigue intercepts (blocks) all directed paths from Lung_cancer to C

In [13]:
identified_estimand.backdoor_variables

{'backdoor1': ['Yellow_Fingers',
  'Anxiety',
  'Genetics',
  'Smoking',
  'Allergy',
  'Attention_Disorder',
  'Peer_Pressure'],
 'backdoor2': ['Genetics'],
 'backdoor3': ['Attention_Disorder'],
 'backdoor': ['Genetics']}

### Correlation / Dependence Analysis

In [114]:
corr = df.corr()
cmap = cmap=sns.diverging_palette(5, 250, as_cmap=True)
corr.style.background_gradient(cmap, axis=1)\
.set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
.set_precision(3)

,Smoking,Yellow_Fingers,Anxiety,Peer_Pressure,Genetics,Attention_Disorder,Born_an_Even_Day,Car_Accident,Fatigue,Allergy,Coughing,Lung_cancer
Smoking,1.000,0.775,0.401,0.149,0.010,0.004,-0.022,0.051,0.163,0.036,0.262,0.491
Yellow_Fingers,0.775,1.000,0.308,0.115,-0.004,0.007,-0.006,0.049,0.125,0.047,0.207,0.377
Anxiety,0.401,0.308,1.000,0.003,0.006,-0.015,-0.038,0.031,0.051,0.018,0.137,0.190
Peer_Pressure,0.149,0.115,0.003,1.000,0.020,0.015,-0.007,0.024,0.028,-0.005,0.050,0.057
Genetics,0.010,-0.004,0.006,0.020,1.000,0.269,-0.022,0.146,0.100,-0.008,0.137,0.228
Attention_Disorder,0.004,0.007,-0.015,0.015,0.269,1.000,-0.021,0.303,0.033,0.020,0.054,0.068
Born_an_Even_Day,-0.022,-0.006,-0.038,-0.007,-0.022,-0.021,1.000,-0.031,-0.028,-0.033,-0.028,-0.012
Car_Accident,0.051,0.049,0.031,0.024,0.146,0.303,-0.031,1.000,0.463,0.038,0.205,0.169
Fatigue,0.163,0.125,0.051,0.028,0.100,0.033,-0.028,0.463,1.000,0.094,0.460,0.369
Allergy,0.036,0.047,0.018,-0.005,-0.008,0.020,-0.033,0.038,0.094,1.000,0.307,-0.026


In [115]:
# %pip install hyppo
# %pip install pingouin
# %pip install conditional_independence

In [116]:
x = df[TREATMENT].values.astype(float)
x_re = x.reshape((2000, 1))
y = df[OUTCOME].values.astype(float)
y_re = y.reshape((2000, 1))

In [117]:
print('Pearson\'s Correlation: %.3f' % np.corrcoef(x, y)[0, 1])

Pearson's Correlation: 0.369


In [118]:
# from hyppo.independence import Hsic

# c_h = Hsic().test(x, y)
# print(c_h)

In [119]:
def rbf_dot(pattern1, pattern2, deg):
	size1 = pattern1.shape
	size2 = pattern2.shape

	G = np.sum(pattern1*pattern1, 1).reshape(size1[0],1)
	H = np.sum(pattern2*pattern2, 1).reshape(size2[0],1)

	Q = np.tile(G, (1, size2[0]))
	R = np.tile(H.T, (size1[0], 1))

	H = Q + R - 2* np.dot(pattern1, pattern2.T)

	H = np.exp(-H/2/(deg**2))

	return H


def hsic_gam(X, Y, alph = 0.5):
	"""
	X, Y are numpy vectors with row - sample, col - dim
	alph is the significance level
	auto choose median to be the kernel width
	"""
	n = X.shape[0]

	# ----- width of X -----
	Xmed = X

	G = np.sum(Xmed*Xmed, 1).reshape(n,1)
	Q = np.tile(G, (1, n) )
	R = np.tile(G.T, (n, 1) )

	dists = Q + R - 2* np.dot(Xmed, Xmed.T)
	dists = dists - np.tril(dists)
	dists = dists.reshape(n**2, 1)

	width_x = np.sqrt( 0.5 * np.median(dists[dists>0]) )
	# ----- -----

	# ----- width of X -----
	Ymed = Y

	G = np.sum(Ymed*Ymed, 1).reshape(n,1)
	Q = np.tile(G, (1, n) )
	R = np.tile(G.T, (n, 1) )

	dists = Q + R - 2* np.dot(Ymed, Ymed.T)
	dists = dists - np.tril(dists)
	dists = dists.reshape(n**2, 1)

	width_y = np.sqrt( 0.5 * np.median(dists[dists>0]) )
	# ----- -----

	bone = np.ones((n, 1), dtype = float)
	H = np.identity(n) - np.ones((n,n), dtype = float) / n

	K = rbf_dot(X, X, width_x)
	L = rbf_dot(Y, Y, width_y)

	Kc = np.dot(np.dot(H, K), H)
	Lc = np.dot(np.dot(H, L), H)

	testStat = np.sum(Kc.T * Lc) / n

	varHSIC = (Kc * Lc / 6)**2

	varHSIC = ( np.sum(varHSIC) - np.trace(varHSIC) ) / n / (n-1)

	varHSIC = varHSIC * 72 * (n-4) * (n-5) / n / (n-1) / (n-2) / (n-3)

	K = K - np.diag(np.diag(K))
	L = L - np.diag(np.diag(L))

	muX = np.dot(np.dot(bone.T, K), bone) / n / (n-1)
	muY = np.dot(np.dot(bone.T, L), bone) / n / (n-1)

	mHSIC = (1 + muX * muY - muX - muY) / n

	al = mHSIC**2 / varHSIC
	bet = varHSIC*n / mHSIC

	thresh = gamma.ppf(1-alph, al, scale=bet)[0][0]

	return (testStat, thresh)

In [120]:
c_h = hsic_gam(x_re, y_re, alph=0.05)
print(c_h)

(16.920357647960223, 0.29834572066385023)


In [121]:
import pingouin as pg

In [122]:
pg.corr(x, y)

,n,r,CI95%,p-val,BF10,power
pearson,2000,0.368653,"[0.33, 0.41]",2.035202e-65,5.744e+61,1.0


In [127]:
pg.partial_corr(data=df, x = TREATMENT, y = OUTCOME, covar = 'Coughing').round(5)

,n,r,CI95%,p-val
pearson,2000,0.17237,"[0.13, 0.21]",0.0


In [128]:
import conditional_independence.ci_tests.nonparametric.hsic as cond_hsic

res = cond_hsic.hsic_test(df.to_numpy(), df.columns.get_loc(TREATMENT), 
                          df.columns.get_loc(OUTCOME), df.columns.get_loc('Coughing'))
print(res)

{'statistic': 0.001217202188057207, 'p_value': 4.440892098500626e-16, 'reject': True, 'mean_approx': 2.089839045117442e-05, 'var_approx': 7.606018338288321e-10}


### Identify Estimands

In [10]:
df_t = df.copy()
for column in df_t.columns:
    df_t = df_t.astype({column: 'bool'})
df_t.head()

,Smoking,Yellow_Fingers,Anxiety,Peer_Pressure,Genetics,Attention_Disorder,Born_an_Even_Day,Car_Accident,Fatigue,Allergy,Coughing,Lung_cancer
0,False,False,True,False,False,True,False,True,False,True,False,False
1,False,True,False,False,False,False,True,False,True,False,True,True
2,True,True,True,False,True,True,True,True,True,True,True,True
3,False,False,False,True,False,False,True,False,False,False,False,False
4,True,True,True,False,False,True,True,True,True,False,False,True


In [14]:
model = CausalModel(
                data = df_t,
                treatment = TREATMENT,
                outcome = OUTCOME,
                graph = gml_graph
        )

In [15]:
# identify estimands

identified_estimand = model.identify_effect()
print(identified_estimand)

Estimand type: nonparametric-ate

### Estimand : 1
Estimand name: backdoor
Estimand expression:
      d                                           
──────────────(Expectation(Car_Accident|Genetics))
d[Lung_cancer]                                    
Estimand assumption 1, Unconfoundedness: If U→{Lung_cancer} and U→Car_Accident then P(Car_Accident|Lung_cancer,Genetics,U) = P(Car_Accident|Lung_cancer,Genetics)

### Estimand : 2
Estimand name: iv
Estimand expression:
Expectation(Derivative(Car_Accident, [Smoking])*Derivative([Lung_cancer], [Smo
king])**(-1))
Estimand assumption 1, As-if-random: If U→→Car_Accident then ¬(U →→{Smoking})
Estimand assumption 2, Exclusion: If we remove {Smoking}→{Lung_cancer}, then ¬({Smoking}→Car_Accident)

### Estimand : 3
Estimand name: frontdoor
Estimand expression:
Expectation(Derivative(Car_Accident, [Fatigue])*Derivative([Fatigue], [Lung_ca
ncer]))
Estimand assumption 1, Full-mediation: Fatigue intercepts (blocks) all directed paths from Lung_cancer to C

#### Backdoor

In [132]:
identifier = model.identifier
identifier.identify_backdoor(identifier.treatment_name, identifier.outcome_name)

[{'backdoor_set': set()},
 {'backdoor_set': ('Yellow_Fingers',
   'Smoking',
   'Genetics',
   'Allergy',
   'Attention_Disorder',
   'Anxiety',
   'Peer_Pressure')},
 {'backdoor_set': ('Yellow_Fingers',)},
 {'backdoor_set': ('Smoking',)},
 {'backdoor_set': ('Genetics',)},
 {'backdoor_set': ('Allergy',)},
 {'backdoor_set': ('Attention_Disorder',)},
 {'backdoor_set': ('Anxiety',)},
 {'backdoor_set': ('Peer_Pressure',)}]

In [25]:
identified_estimand.get_backdoor_variables()

['Genetics']

#### Experiment: Backdoor by hand

In [16]:
from dowhy.causal_identifiers import backdoor
nx.descendants(G, TREATMENT) | {TREATMENT}

H = G.to_undirected()
all_possible_paths = [x for x in nx.all_simple_paths(H, TREATMENT, OUTCOME)]
all_possible_paths

[['Lung_cancer', 'Genetics', 'Attention_Disorder', 'Car_Accident'],
 ['Lung_cancer', 'Coughing', 'Fatigue', 'Car_Accident'],
 ['Lung_cancer', 'Fatigue', 'Car_Accident']]

In [17]:
def get_adjustment_variables(G, paths):

    adjustment_variables = pd.DataFrame(columns=['path', 'colliders_desc', 'non_colliders'])

    for path in paths:
        colliders_desc = np.array([])
        non_colliders = []
        path_len = len(path)

        for node0, node1, node2 in zip(path[0:path_len-2], path[1:path_len-1], path[2:]):
        # if there is an arrow pointing into node1 from both sides on the path, it is a collider
            if G.has_edge(node0, node1) and G.has_edge(node2, node1):
                colliders_desc = np.append(colliders_desc, list(nx.descendants(G,node1)) + [node1]) # so we add it (and all its descendants) to the list
        # we flatten the list of list
        colliders_desc = colliders_desc.flatten()
        adjustment_variables = adjustment_variables.append({'path':path, 'colliders_desc': colliders_desc, 
                                                    'non_colliders': non_colliders}, ignore_index=True)
    
    return adjustment_variables

In [18]:
adj = get_adjustment_variables(G=G, paths=all_possible_paths)
adj.head()

,path,colliders_desc,non_colliders
0,"[Lung_cancer, Genetics, Attention_Disorder, Ca...",[],[]
1,"[Lung_cancer, Coughing, Fatigue, Car_Accident]",[],[]
2,"[Lung_cancer, Fatigue, Car_Accident]",[],[]


#### Frontdoor

In [24]:
identified_estimand.get_frontdoor_variables()

['Fatigue']

#### Instrumental Variables

In [26]:
identified_estimand.get_instrumental_variables()

['Smoking']

### Causal Effect Estimation

In [136]:
# estimate causal effect

estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.linear_regression")
print("Causal Estimate is " + str(estimate.value))

Causal Estimate is 0.36255807847398913


In [137]:
dowhy.plotter.plot_causal_effect(estimate, df[TREATMENT], df[OUTCOME])

In [139]:
estimate.get_confidence_intervals()

In [140]:
# using the backdoor adjustment set and propensity score stratification
causal_estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_matching")
print(causal_estimate)
print("Causal Estimate is " + str(causal_estimate.value))

Exception: No common causes/confounders present. Propensity score based methods are not applicable

In [141]:
# using the instrumental variables
causal_estimate = model.estimate_effect(identified_estimand,
        method_name="iv.instrumental_variable")
print(causal_estimate)
print("Causal Estimate is " + str(causal_estimate.value))

LinAlgError: Singular matrix

### Causal Discovery